In [8]:
from sklearn import datasets
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
import matplotlib.pyplot as plt

import torch
from torch.autograd import Variable
from torch import nn
from torch.nn import functional as F

from helpers import *

%load_ext autoreload
%autoreload 2

torch.manual_seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading the data

In [9]:
import dlc_bci as bci
train_input , train_target = bci.load(root = './data_bci')
print(str(type(train_input)), train_input.size()) 
print(str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root = './data_bci', train = False)
print(str(type(test_input)), test_input.size()) 
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])


## Pre-processing

In [10]:
# normalize mean to 0
train_input.sub_(train_input.mean())
test_input.sub_(test_input.mean())

# normalize variance to 1
train_input.div_(train_input.std())
test_input.div_(test_input.std())

print('done')

done


## Linear Regression

In [ ]:
lr = linear_model.LinearRegression()

X_train = train_input.view(train_input.size(0), -1)
X_test = test_input.view(test_input.size(0), -1)

# cross_val_predict returns an array of the same size as `y` where each entry
# is a prediction obtained by cross validation:
lr.fit(X=X_train, y=train_target)
preds = lr.predict(X_test)

preds = torch.FloatTensor(preds)

In [ ]:
preds = discrete_predictions(preds)
accuracy = compute_accuracy(test_target, preds)
print('Accuracy ' + str(accuracy*100) + '%')

## Neural Network

In [ ]:
# load data
train_input , train_target = bci.load(root = './data_bci')
test_input , test_target = bci.load(root = './data_bci', train = False)

In [ ]:
train_input2 = train_input[0:300]
train_target2 = train_target[0:300]

In [104]:
class Net(nn.Module):
    def __init__(self, nb_hidden=64):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(640, 2)

    def forward(self, x):
        #print("======")
        print(x.size())
        x = F.tanh(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        print(x.size())
        x = F.tanh(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        print(x.size())
        x = F.tanh(self.fc1(x.view(-1, 640)))
        print(x.size())
        #print("======")
        return x

In [ ]:
my_cool_net = Net()
train_model(my_cool_net, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4, nb_epochs=50, learning_rate=1e-2, verbose=True)

#### Compute error rate

In [ ]:
nb_errors = compute_nb_errors(my_cool_net, Variable(test_input.view(-1, 1, 28, 50)), Variable(test_target), 4)

In [ ]:
nb_errors/test_input.size(0)

In [ ]:
train_errors = compute_nb_errors(my_cool_net, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4)
train_errors / train_input.size(0)

### Grid search on model parameters

In [ ]:
step_sizes = [0.01, 0.005, 0.0025, 0.001, 0.0005]
nb_iters_ = [25, 50, 100, 250, 500]

test_errors = []
train_errors = []

for lr in step_sizes:
    test_errs = []
    train_errs = []
    for nb_iters in nb_iters_:
        # reset and train the network
        my_net = Net()
        train_model(my_net, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), mini_batch_size=4, nb_epochs=nb_iters, learning_rate=lr)
        
        # compute the number of errors
        n_train_errors = compute_nb_errors(my_net, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4)
        n_test_errors = compute_nb_errors(my_net, Variable(test_input.view(-1, 1, 28, 50)), Variable(test_target), 4)
        
        train_error = n_train_errors / train_input.size(0)
        test_error = n_test_errors / test_input.size(0)
        
        test_errs.append(test_error)
        train_errs.append(train_error)
        
        print('step size: ' + str(lr) + ' nb epochs: ' + str(nb_iters) + ' train error: ' + str(train_error) + ' test error: ' + str(test_error))
    test_errors.append(test_errs)
    train_errors.append(train_errs)

## LSTM 

In [203]:
class Net2(nn.Module):
    def __init__(self, nb_hidden=64):
        super(Net2, self).__init__()
        self.inp = nn.Linear(50, 32)
        self.lstm = nn.LSTM(32, 32, 2, dropout=0.05)
        #self.lstm2 = nn.LSTM(32, 32, 2, dropout=0.2)
        self.out = nn.Linear(28*32, 2)

    def forward(self, x, hc=None):
        #print("======")
        #print(x.size())
        #x = F.tanh(F.max_pool2d(self.lstm2(x), kernel_size=2, stride=2))
        x = F.tanh(self.inp(x))
        #print(x.size())
        x, hidden = self.lstm(x.squeeze(1), hc)
        x = F.tanh(x)
        #print(x.size())
        #x, hidden = self.lstm2(x, hc)
        #print(x.size())
        x = F.tanh(self.out(x.view(-1, 28*32)))
        #print(x.size())
        #print("======")
        return x

In [204]:
model = Net2()
nb_epochs=100
mini_batch_size=4
learning_rate=0.1
verbose=True
train_in = Variable(train_input.view(-1, 1, 28, 50))
train_targ = Variable(train_target)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = learning_rate)
for e in range(0, nb_epochs):
    sum_loss = 0
    for b in range(0, train_in.size(0), mini_batch_size):
        output = model(train_in.narrow(0, b, mini_batch_size))
        #print(train_targ.narrow(0, b, mini_batch_size).size())
        #print(output.size())
        loss = criterion(output, train_targ.narrow(0, b, mini_batch_size))
        sum_loss = sum_loss + loss.data[0]
        model.zero_grad()
        loss.backward()
        optimizer.step()
    if verbose:
        print(e, sum_loss)

0 55.73849070072174
1 55.38568425178528
2 55.21029359102249
3 55.10095900297165
4 54.998302578926086
5 54.86130082607269
6 54.700638473033905
7 54.42222648859024
8 54.12572079896927
9 53.78873461484909
10 53.358709782361984
11 52.860850274562836
12 52.26632231473923
13 51.40733340382576
14 50.350856959819794
15 49.017218232154846
16 47.77862358093262
17 46.44073760509491
18 45.236290737986565
19 44.11910480260849
20 43.10380092263222
21 41.45946538448334
22 40.12917344272137
23 38.74941794574261
24 38.039336293935776
25 37.47816778719425
26 36.333162158727646
27 35.75167179107666
28 34.646584168076515
29 33.99380961060524
30 32.875477120280266
31 32.5782028734684
32 31.80992814898491
33 31.27399778366089
34 29.31551556289196
35 29.736986055970192
36 28.919235825538635
37 28.256519332528114
38 28.86168311536312
39 26.187440767884254
40 26.5390802025795
41 24.347222849726677
42 24.032073840498924
43 23.85285922884941
44 22.041225388646126
45 23.528141885995865
46 22.044113844633102
47 20

In [205]:
nb_errors = compute_nb_errors(model, Variable(test_input.view(-1, 1, 28, 50)), Variable(test_target), 4)
1-nb_errors/test_input.size(0)

0.69

In [206]:
train_errors = compute_nb_errors(model, Variable(train_input.view(-1, 1, 28, 50)), Variable(train_target), 4)
1 - train_errors / train_input.size(0)

0.9841772151898734

In [143]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torch import optim
import numpy as np
import math, random

# Generating a noisy multi-sin wave 

def sine_2(X, signal_freq=60.):
    return (np.sin(2 * np.pi * (X) / signal_freq) + np.sin(4 * np.pi * (X) / signal_freq)) / 2.0

def noisy(Y, noise_range=(-0.05, 0.05)):
    noise = np.random.uniform(noise_range[0], noise_range[1], size=Y.shape)
    return Y + noise

def sample(sample_size):
    random_offset = random.randint(0, sample_size)
    X = np.arange(sample_size)
    Y = noisy(sine_2(X + random_offset))
    print(Y.shape)
    print('==========')
    return Y

class SimpleRNN(nn.Module):
    def __init__(self, hidden_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size

        self.inp = nn.Linear(1, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, 2, dropout=0.05)
        self.out = nn.Linear(hidden_size, 1)

    def step(self, input, hidden=None):
        input = self.inp(input.view(1, -1)).unsqueeze(1)
        output, hidden = self.rnn(input, hidden)
        output = self.out(output.squeeze(1))
        return output, hidden

    def forward(self, inputs, hidden=None, force=True, steps=0):
        if force or steps == 0: steps = len(inputs)
        outputs = Variable(torch.zeros(steps, 1, 1))
        for i in range(steps):
            if force or i == 0:
                input = inputs[i]
            else:
                input = output
            output, hidden = self.step(input, hidden)
            outputs[i] = output
        return outputs, hidden

In [ ]:
#lstm_net = Net2()
#train_model(lstm_net, Variable(train_input), Variable(train_target), 4, nb_epochs=50, learning_rate=1e-2, verbose=True)


n_epochs = 100
n_iters = 50
hidden_size = 10

model = SimpleRNN(hidden_size)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

losses = np.zeros(n_epochs) # For plotting

for epoch in range(n_epochs):

    for iter in range(n_iters):
        _inputs = sample(50)
        inputs = Variable(torch.from_numpy(_inputs[:-1]).float())
        targets = Variable(torch.from_numpy(_inputs[1:]).float())

        # Use teacher forcing 50% of the time
        force = random.random() < 0.5
        outputs, hidden = model(inputs, None)

        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        losses[epoch] += loss.data[0]

    if epoch > 0:
        print(epoch, loss.data[0])

In [26]:
import torch
import torch.nn as nn
from torch.autograd import Variable

time_steps = 10
batch_size = 3
in_size = 5
classes_no = 7

model = nn.LSTM(in_size, classes_no, 2)
input_seq = Variable(torch.randn(time_steps, batch_size, in_size))
output_seq, _ = model(input_seq)
last_output = output_seq[-1]

loss = nn.CrossEntropyLoss()
target = Variable(torch.LongTensor(batch_size).random_(0, classes_no-1))
err = loss(last_output, target)
err.backward()